# Precomputation of exposure integral

Now that we implement the effective area as a function of energy and zenith angle, can keep the fit fast by precomputing the exposure factor for each source. This is possible if assuming *known* source positions and redshifts.

For now, we assume that the background has some fixed redshift. This is necessary in order to compute the energy losses in the simulation. This is something that will probably be changed in the next step as we include selection effects and try to do something more realistic. 

## What is the exposure integral?

As part of the simulation and fit, we want to compute the expected number of detected neutrinos from each source component. This can be written as $\bar{N}_k = F_k \epsilon_k$ where $F_k = \frac{Q}{4 \pi d_l(z_k)^2}$ and the exposure factor is 
$$
\epsilon_k = T \frac{(\alpha-1)}{E_{\rm{min}}} \int_{E_{\rm{min}}}^\infty {\rm{d}}E \ A_{\rm{eff}}(E, \varpi_k) 
\Bigg( \frac{(1+z_k)E}{E_{\rm{min}}} \Bigg)^{-\alpha} = T \frac{(\alpha-1)}{E_{\rm{min}}} I_k, 
$$
where $k$ is the index for each source (see likelihood derivation for details).

Here we will compute $I_k$ for a range of $\alpha$ so that we can perform fast linear interpolation in the fit for unknown $\alpha$.

In [ ]:
import numpy as np
import h5py
from scipy.interpolate import RectBivariateSpline

from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

### Effective area

Use the module provided by Martin & Hans to produce a spline with scipy (same code, just a few edits to run in python 3). Also, use a quick spline implementation to tets out the case for muon tracks.

In [2]:
from aeff_calculator.aeff_calculator import effective_area_tracks, effective_area_cascades

aeff_tracks = effective_area_tracks()
aeff_cascades = effective_area_cascades("nue_CC")

... creating effective area spline for interaction nue_CC , done!


### Sources

Use a set of sources sampled as having redshift uniform in comoving volume for a $\Lambda CDM$ universe.

In [3]:
with h5py.File('../data/test_SFR_pop.h5', 'r') as f:
    Ns = f['output/Ns'].value
    redshift = f['output/redshift'][()]
    position = f['output/position'][()]
    
# Select sources within redshift zth to mimic catalogue
zth = 1
selection = np.where(redshift < zth)[0]
Ns = len(selection)
redshift = np.array([redshift[_] for _ in selection])
position = np.array([position[_] for _ in selection])

# Add background redshift
source_redshift = list(redshift) 
bg_redshift = zth

/Users/fran/.virtualenvs/bayes/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


### Computation

In [4]:
from exposure_integral import ExposureIntegral

/Users/fran/.virtualenvs/bayes/lib/python3.7/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


#### Cascades

In [ ]:
# Define minimum energy
Emin = 6.0E4 # GeV

exposure_integral = ExposureIntegral(source_redshift, position, bg_redshift, 
                                     aeff_cascades, Emin,
                                    filename='data/precomputed_integral_cascades_6e4.h5',
                                     n_points=100)

In [ ]:
fig, ax = plt.subplots()
for I_k in exposure_integral.integral_grid[:-1]:
    ax.scatter(exposure_integral.alpha_grid, I_k)
ax.scatter(exposure_integral.alpha_grid, exposure_integral.integral_grid[-1], 
           color='k')

#### Tracks

In [5]:
# Define minimum energy
Emin = 1.0E5 # GeV

exposure_integral = ExposureIntegral(source_redshift, position, bg_redshift, 
                                     aeff_tracks, Emin,
                                    filename='data/precomputed_integral_tracks_1e5.h5',
                                     n_points=100)

/Users/fran/projects/astroparticle/hierarchical_nu/dev/statistical_model/4_tracks_and_cascades/exposure_integral.py:73: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  args=(position, z, alpha))


/Users/fran/.virtualenvs/bayes/lib/python3.7/site-packages/scipy/integrate/quadpack.py:860: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  **opt)


49632359.60159601 2883.551161272808
47148747.68204771 2790.85177189816
44769945.63681918 2692.442646032061
42492892.04046651 2592.9834191692935
40314536.60093746 2490.3750565654336
38231844.93785537 2392.0982923580814
36241802.1745401 2299.6686224925434
34341416.78799444 2209.6172543451903
32527724.685529035 2120.254607796167
30797792.8431993 2035.8083330572263
29148722.26937317 1955.4990174745762
27577651.75938463 1878.916995111682
26081760.57703619 1805.8233540637193
24658271.916699532 1736.012656436822
23304455.292440694 1669.2749382595794
22017629.207364757 1605.9640314759126
20795163.38774704 1545.7193770424856
19634481.336276665 1488.2572786769706
18533061.962353215 1433.1485125549043
17488441.51370725 1323.95971044644
16498215.207914045 1289.9902639641127
15560038.628559805 1256.7039819738677
14671708.05049321 1189.6775367344446
13830766.112216828 1158.8621532195095
13035293.217759257 1128.6522634971793
12283120.43261229 1099.0381792822561
11572211.48437215 1070.0102296694213
10

In [ ]:
fig, ax = plt.subplots()
for I_k in exposure_integral.integral_grid[:-1]:
    ax.scatter(exposure_integral.alpha_grid, I_k)
ax.scatter(exposure_integral.alpha_grid, exposure_integral.integral_grid[-1], 
           color='k')